In [5]:
import matplotlib.pyplot as plt
import numpy as np
from utils.objects import GenerationClass
from utils.config import color_maps, ycb_heights
import os
from glob import glob
from scipy import sparse
from PIL import Image
import json
from mani_skill2.envs.pick_and_place.pick_cube import PickCubeEnv
from mani_skill2.utils.sapien_utils import look_at
from mani_skill2.sensors.camera import CameraConfig
from mani_skill2.utils.registration import register_env
from pathlib import Path
import gymnasium as gym
from sapien.core import Pose
import numpy as np
from transforms3d.euler import euler2quat, quat2euler
from utils.config import ycb_heights, color_maps
from glob import glob
from scipy import sparse
from PIL import Image
import json
import os

from IPython.display import clear_output
# save_dir = "../final_data/"
# move_one_dir = os.path.join(save_dir, "move")
# move_cube_dir = os.path.join(move_one_dir, "cube")
# move_ycb_dir = os.path.join(move_one_dir, "ycb")
# os.makedirs(move_one_dir, exist_ok=True)
# os.makedirs(move_cube_dir, exist_ok=True)
# os.makedirs(move_ycb_dir, exist_ok=True)

In [6]:
model_paths = sorted(glob("../data/mani_skill2_ycb/models/*"), key=lambda x: int(x.split('/')[-1][:3]))
gen = GenerationClass(model_paths)

In [7]:
# n = 20
# gen.generate(1, "place_object_in_direction", n)
# gen.generate(3, "place_object_in_direction", n)
# gen.generate(4, "place_object_in_direction", n)

# gen.generate(2, "place_object_on_another", n)
# gen.generate(3, "place_object_on_another", n)
# gen.generate(4, "place_object_on_another", n)

# gen.generate(3, "place_object_in_between", n)

# gen.generate(3, "remove_object", n)

# gen.generate(3, "order_by_color", n)

# gen.generate(2, "move_object", n)

# gen.generate(3, "move_object_toward_another", n)

In [ ]:
### imported 3d objects


@register_env("CustomEnv-v0", max_episode_steps=200, override=True)
class CustomEnv(PickCubeEnv):
    def _load_actors(self):
        self._add_ground(render=self.bg_name is None)
        objs = []
        builder = self._scene.create_actor_builder()

        name = "eiffel_tower"

        filepath = f"/home/mkim/Nserver/mix-lab/maniskill2_dev/models/{name}/mesh.obj"
        scale = 0.5
        scale *= self.cube_half_size / 0.01887479572529618
        builder.add_multiple_collisions_from_file(
            filename=filepath, scale=scale, density=1000
        )
        builder.add_visual_from_file(filename=filepath, scale=scale)
        self.obj = builder.build_static(name=name)

        self.objs = objs
        self.goal_site = self._build_sphere_site(self.goal_thresh)

    def _register_cameras(self):
        pose = look_at([-1, 0, 1], [0, 0, 0])
        return CameraConfig(
            "base_camera", pose.p, pose.q, 512, 512, 1.2, 0.01, 10
        )
    

env = gym.make("CustomEnv-v0", obs_mode="rgbd", camera_cfgs={"add_segmentation": True})

obs, _ =env.reset()


env.unwrapped.get_articulations()[0].set_pose(Pose([0, 0, -2], [1, 0, 0, 0]))
# quat = euler2quat(np.random.uniform(-np.pi*2, np.pi*2), np.random.uniform(-np.pi*2, np.pi*2), np.random.uniform(-np.pi*2, np.pi*2))
quat = euler2quat(np.pi/2, 0, np.random.uniform(np.pi*3/4, np.pi*5/4))#np.random.uniform(-np.pi*2, np.pi*2))
# quat = np.array([ 0.37677702, -0.25098675, -0.80399704, -0.38553014])
env.obj.set_pose(Pose([-0.5, 0, 0.3], quat))
# for i in range(10):
obs, _, _, _, _ = env.step(np.zeros(len(env.action_space.sample())))

img = obs['image']["base_camera"]["rgb"]
mask = obs['image']["base_camera"]["Segmentation"][..., :3].copy()
mask[mask != env.obj.id-2] = 0
mask = np.amax(mask.astype(bool), axis=2)

img[mask] += 50
# img[mask] += 25
img = np.clip(img, 0, 255)

clear_output(wait=True)
plt.imshow(img)
plt.show()